In [2]:
# --- Euclidean donut buffers for Elizabeth line stations (EPSG:27700) ---
import geopandas as gpd
import pyogrio
from pathlib import Path
import pandas as pd

# ======= 输入路径（按你的目录）=======
P_STATIONS = "data/interim/tfl_elizabeth_line_stations_2022_27700.gpkg"
# 路由文件对欧氏缓冲不需要，但保留变量方便以后扩展
P_ROUTE    = "data/interim/tfl_elizabeth_line_route_2022_27700.gpkg"

# ======= 输出路径 =======
OUT_GPKG   = "data/clean/elizabeth_station_rings_27700.gpkg"
OUT_LAYER  = "elizabeth_station_rings_27700"

# ------- 工具函数 -------
def read_first_layer(path):
    layer = pyogrio.list_layers(path)[0][0]
    return gpd.read_file(path, layer=layer, engine="pyogrio")

def ensure_27700(gdf):
    # 若不是投影坐标，统一到 EPSG:27700（单位=米）
    if gdf.crs is None:
        raise ValueError("输入数据缺少 CRS（坐标参考系）。请确认为 EPSG:27700。")
    if getattr(gdf.crs, "to_epsg", lambda: None)() != 27700:
        gdf = gdf.to_crs(27700)
    return gdf

# ------- 读取数据 -------
stns = ensure_27700(read_first_layer(P_STATIONS)).reset_index(drop=True)

# 识别一个名字列；若没有则用顺序号
name_col = next((c for c in ["NAME","FULL_NAME","FullName","Station","station",
                             "station_name","name","TITLE","Title"] if c in stns.columns), None)
stns["station_id"] = (stns.get("OBJECTID", pd.Series(range(len(stns))))).astype(str)
stns["station_name"] = stns[name_col].astype(str) if name_col else ("station_" + (stns.index+1).astype(str))

# ------- 生成欧氏缓冲与甜甜圈 -------
# 注意：几何层面生成的是闭集；“前闭后开”的逻辑在后续点入带判断时实现。
# 这里我们仍按标准做差，保证带与带之间不重叠。
buf100 = stns.geometry.buffer(100)
buf400 = stns.geometry.buffer(400)
buf800 = stns.geometry.buffer(800)

ring0_100   = buf100
ring100_400 = buf400.difference(buf100)
ring400_800 = buf800.difference(buf400)

# ------- 组装为一个图层 -------
def make_ring_gdf(geom_series, band_code, lo, hi, interval):
    return gpd.GeoDataFrame({
        "station_id":   stns["station_id"].values,
        "station_name": stns["station_name"].values,
        "band":         [band_code]*len(stns),
        "dist_min_m":   [lo]*len(stns),
        "dist_max_m":   [hi]*len(stns),
        "interval":     [interval]*len(stns),
        "geometry":     geom_series
    }, crs=stns.crs)

g0_100   = make_ring_gdf(ring0_100,   "b0_100",     0, 100,  "[0,100)")
g100_400 = make_ring_gdf(ring100_400, "b100_400", 100, 400, "[100,400)")
g400_800 = make_ring_gdf(ring400_800, "b400_800", 400, 800, "[400,800)")

rings = pd.concat([g0_100, g100_400, g400_800], ignore_index=True)

# 可选：去除空几何（理论上不该出现，但稳妥）
rings = rings[~rings.geometry.is_empty & rings.geometry.notna()].copy()

# ------- 写出 GPKG（覆盖同名文件与图层） -------
Path(OUT_GPKG).parent.mkdir(parents=True, exist_ok=True)
# 若存在则先删除以避免堆叠旧层
try:
    Path(OUT_GPKG).unlink()
except FileNotFoundError:
    pass

pyogrio.write_dataframe(rings, OUT_GPKG, layer=OUT_LAYER, driver="GPKG")

print(f"Done. Wrote {len(rings)} features to {OUT_GPKG} (layer={OUT_LAYER}).")
print(rings.groupby('band')[['dist_min_m','dist_max_m']].first())


Done. Wrote 123 features to data/clean/elizabeth_station_rings_27700.gpkg (layer=elizabeth_station_rings_27700).
          dist_min_m  dist_max_m
band                            
b0_100             0         100
b100_400         100         400
b400_800         400         800
